<a href="https://colab.research.google.com/github/rosshalpin/AdvancedML/blob/main/AdvancedML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive/AdvancedML/

/
Mounted at /content/gdrive
/content/gdrive/My Drive/AdvancedML


In [ ]:
!pip install tweepy --upgrade
!pip install black[jupyter]

     |████████████████████████████████| 76 kB 2.8 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 1.4 MB 5.3 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 843 kB 44.3 MB/s 
     |████████████████████████████████| 793 kB 43.9 MB/s 
     |████████████████████████████████| 380 kB 35.5 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2

In [ ]:
# !black /mydrive/Colab\ Notebooks/AdvancedML.ipynb

In [ ]:
%cd /mydrive/AdvancedML/

/content/gdrive/My Drive/AdvancedML


In [ ]:
import os, tweepy

with open('bearer.txt', 'r') as file:
  BEARER_TOKEN = file.read().rstrip()

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [ ]:
tweet_paginator = tweepy.Paginator(client.search_recent_tweets,
                              query="Ukraine Immigrants -is:retweet lang:en",
                              tweet_fields=[
                                "created_at", 
                                "possibly_sensitive", 
                                "public_metrics",
                              ],
                              place_fields=[
                                "full_name",
                                "geo",
                                "country",
                                "country_code"    
                              ],
                              expansions="geo.place_id",
                              max_results=20,
                              limit=10)

In [ ]:
responses = []
for resp in tweet_paginator:
  responses.append(resp)

In [ ]:
found_places = {}

for resp in responses:
  places = resp.includes.get('places')
  if places:
    for place in places:
      found_places[place.id] = place

print(found_places)

{'de55c30bd4263e8a': <Place id=de55c30bd4263e8a full_name=Apollo Beach, FL>, 'd374fb61a20fb74f': <Place id=d374fb61a20fb74f full_name=Nevada, USA>}


In [ ]:
import pandas as pd
tweets = []

def pre_process_text(text: str) -> str:
    text = text.replace("\n", " ").replace("\t", " ")
    return text.strip()

for resp in responses:
  for tweet in resp.data:
    if tweet.geo:
      place_id = tweet.geo.get('place_id')
      coords = found_places[place_id].geo.get('bbox')
      lon = (coords[0]+coords[2])/2
      lat = (coords[1]+coords[3])/2
      new_tweet = [
          found_places[place_id].full_name,
          f"{lat:.2f},{lon:.2f}",
          tweet.created_at, 
          tweet.possibly_sensitive, 
          tweet.public_metrics.get('retweet_count'), 
          tweet.public_metrics.get('reply_count'),
          tweet.public_metrics.get('like_count'),
          tweet.public_metrics.get('quote_count'),
          pre_process_text(tweet.text),
      ]
    else:
      new_tweet = [
        None,
        None,
        tweet.created_at,
        tweet.possibly_sensitive,
        tweet.public_metrics.get('retweet_count'), 
        tweet.public_metrics.get('reply_count'),
        tweet.public_metrics.get('like_count'),
        tweet.public_metrics.get('quote_count'),
        pre_process_text(tweet.text),
      ]
    tweets.append(new_tweet)

In [ ]:
columns = ["place", "coords", "created_at", "possibly_sensitive", "retweet_count", "reply_count", "like_count", "quote_count", "text"]
df = pd.DataFrame(tweets, columns=columns)

In [ ]:
df.head()

,place,coords,created_at,possibly_sensitive,retweet_count,reply_count,like_count,quote_count,text
0,None,None,2022-03-21 21:19:21+00:00,False,0,0,0,0,The NATO alliance is doing everything n it pow...
1,None,None,2022-03-21 21:11:05+00:00,False,2,0,2,0,The world mourns &amp; shows support 4 #Ukrain...
2,None,None,2022-03-21 21:08:07+00:00,False,1,0,1,1,@WhiteHouse @WHCOVIDResponse @LDN_gov @CDCgov ...
3,None,None,2022-03-21 21:03:26+00:00,False,0,0,0,0,@NoscereB612 @ElijahJwon @venusinreverse @hunn...
4,None,None,2022-03-21 20:42:02+00:00,False,0,0,0,0,@SYeezy05 @paget_old which one in particular? ...


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 12.4 MB/s 
     |████████████████████████████████| 6.5 MB 65.4 MB/s 
     |████████████████████████████████| 67 kB 7.3 MB/s 
     |████████████████████████████████| 895 kB 60.2 MB/s 
     |████████████████████████████████| 596 kB 59.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task = "sentiment"
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels = []
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode("utf-8").split("\n")
    csvreader = csv.reader(html, delimiter="\t")
labels = [row[1] for row in csvreader if len(row) > 1]

# TF
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
# text = "I love crackers"
# encoded_input = tokenizer(text, return_tensors="tf")
# output = model(encoded_input)
# scores = output[0][0].numpy()
# scores = softmax(scores)
# ranking = np.argsort(scores)
# ranking = ranking[::-1]
# # for i in range(scores.shape[0]):
# #   l = labels[ranking[i]]
# #   s = scores[ranking[i]]
# #   print(f"{i+1}) {l} {np.round(float(s), 4)}")

# results = {i+1: scores[ranking[i]] for i in range(scores.shape[0])}
# max_key = max(results, key=results.get)
# print(max_key)

In [ ]:
# df = df.reset_index()  # make sure indexes pair with number of rows
sentiment = []
for index, row in df.iterrows():
  text = row["text"]
  encoded_input = tokenizer(text, return_tensors="tf")
  output = model(encoded_input)
  scores = output[0][0].numpy()
  scores = softmax(scores)
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  sentiment.append(str(labels[ranking[0]]))

In [ ]:
df["sentiment"] = sentiment

In [ ]:
df

,place,coords,created_at,possibly_sensitive,retweet_count,reply_count,like_count,quote_count,text,sentiment
0,None,None,2022-03-21 21:19:21+00:00,False,0,0,0,0,The NATO alliance is doing everything n it pow...,negative
1,None,None,2022-03-21 21:11:05+00:00,False,2,0,2,0,The world mourns &amp; shows support 4 #Ukrain...,negative
2,None,None,2022-03-21 21:08:07+00:00,False,1,0,1,1,@WhiteHouse @WHCOVIDResponse @LDN_gov @CDCgov ...,neutral
3,None,None,2022-03-21 21:03:26+00:00,False,0,0,0,0,@NoscereB612 @ElijahJwon @venusinreverse @hunn...,negative
4,None,None,2022-03-21 20:42:02+00:00,False,0,0,0,0,@SYeezy05 @paget_old which one in particular? ...,negative
...,...,...,...,...,...,...,...,...,...,...
1425,None,None,2022-03-14 21:52:13+00:00,False,0,0,0,0,HOW MUCH DID CANADA GOV SPEND ON IMMIGRANTS IN...,neutral
1426,None,None,2022-03-14 21:44:08+00:00,False,0,0,0,0,@RepMattGaetz If Ukraine relied on the incomin...,negative
1427,None,None,2022-03-14 21:41:40+00:00,False,0,0,0,0,Maybe the plight of refugees from Ukraine and ...,neutral
1428,None,None,2022-03-14 21:41:36+00:00,False,0,0,0,0,https://t.co/8surV0F8Xk Beautiful love + sicke...,positive


In [ ]:
%cd /mydrive/AdvancedML/
%ls

/content/gdrive/My Drive/AdvancedML
cardiffnlp/


In [ ]:
import time
epoch_time = int(time.time())

df.to_csv("tweets.{}.csv".format(epoch_time), 
          header=list(df.columns), 
          index=False, 
          escapechar="\\", 
          encoding='utf-8',
          quoting=csv.QUOTE_ALL)

In [ ]:
test_df = pd.read_csv(f"tweets.{epoch_time}.csv")

In [ ]:
test_df.head()

,place,coords,created_at,possibly_sensitive,retweet_count,reply_count,like_count,quote_count,text,sentiment
0,NaN,NaN,2022-03-21 21:19:21+00:00,False,0,0,0,0,The NATO alliance is doing everything n it pow...,negative
1,NaN,NaN,2022-03-21 21:11:05+00:00,False,2,0,2,0,The world mourns &amp; shows support 4 #Ukrain...,negative
2,NaN,NaN,2022-03-21 21:08:07+00:00,False,1,0,1,1,@WhiteHouse @WHCOVIDResponse @LDN_gov @CDCgov ...,neutral
3,NaN,NaN,2022-03-21 21:03:26+00:00,False,0,0,0,0,@NoscereB612 @ElijahJwon @venusinreverse @hunn...,negative
4,NaN,NaN,2022-03-21 20:42:02+00:00,False,0,0,0,0,@SYeezy05 @paget_old which one in particular? ...,negative
